In [283]:
#Code Algorithm

import random,itertools
#Generate World:
    #Create Cities, Roads and Evaluate Roads_cost (different than Lenght)
class Edge:

    def __init__(self, origin, dest, cost):
        self.origin = origin
        self.dest = dest
        self.cost = cost
        self.pherom = None

    def getOrigin(self):
        return self.origin

    def getDest(self):
        return self.dest

    def getCost(self):
        return self.cost

    def getPherom(self):
        return self.pherom

    def setPherom(self, pherom):
        self.pherom = pherom

    #Create Graph - Map 
class Graph:

    def __init__(self, num_vertices):
        self.num_vertices = num_vertices # number of vertices/cities
        self.edges = {} # list of edges/roads
        self._neighbours = {} # list of neighbour


    def addEdge(self, origin, dest, cost):
        edge = Edge(origin=origin, dest=dest, cost=cost)
        self.edges[(origin, dest)] = edge
        if origin not in self._neighbours:
            self._neighbours[origin] = [dest]
        else:
            self._neighbours[origin].append(dest)
    
    
    
    def neighbours(self,city):
        return self._neighbours[city]
    
    #@property
    #def edges(origin, dest):
    #    return self.edges[(origin,dest)]

    def getCostEdge(self, origin, dest):
        return self.edges[(origin, dest)].getCost()

    def getPheromEdge(self, origin, dest):
        return self.edges[(origin, dest)].getPherom()

    def setPheromEdge(self, origin, dest, Pherom):
        self.edges[(origin, dest)].setPherom(Pherom)
        
    def pheromReset(self):
        for edge in self.edges:
            x,y = edge
            self.setPheromEdge(x,y,0.01)

    def getCostTour(self, tour):
        cost = 0
        for i in range(self.num_vertices - 1):
            cost += self.getCostEdge(tour[i], tour[i+1])
        # add last cost
        cost += self.getCostEdge(tour[-1], tour[0])
        return cost

    def generateRandom(self):
        for i in range(1, self.num_vertices + 1):
            for j in range(1, self.num_vertices + 1):
                if i != j:
                    cost = random.randint(1, 10)
                    self.addEdge(i, j, cost)




#Generate Colony:
    # Generate Ant
class Ant:

    def __init__(self, graph, city, alpha, beta):
        self.city = city
        self.graph = graph
        self.alpha = alpha
        self.beta = beta
        self._cost = 0
        self.available_cities = [n for n in self.graph.neighbours(self.city)] 
        self.visited_cities = [city] #cities before traveled
        self.start_city = city
        self.solution = []
        
    

   

    #@property
    def getSolution(self):
        return self.solution
    
    #@solution.setter
    #def solution(self,edge):
    #    self._solution.append(edge)

    @property
    def cost(self):
        return self._cost
    
    @cost.setter
    def cost(self,cost):
        self._cost = cost
    
    @property
    def path(self):
        return [edge for edge in self.solution]
    
    def in_nest(self):
        return len(self.visited_cities) - 1 == self.graph.num_vertices
    
    def move(self):
        #Calculate remaining moves
        #Choose one out of the possible moves
        #return chosed move
        
        choices = self.possible_choices()
        choice = self.choose_move(choices)
        return self.make_move(choice)
    
    def possible_choices(self):
        
        return [edge for edge in self.graph.neighbours(self.city) if edge not in self.visited_cities]
    
    def choose_move(self,choices):
        
        if len(choices) == 0:
            return None
        if len(choices) == 1:
            return choices[0]
        
        weights = []
        for move in choices:
            weights.append(self.weigh(self.city,move))
            
        total = sum(weights)
        cumudist = list(itertools.accumulate(weights)) + [total]
        return choices[bisect.bisect(cumudist, random.random() * total)]
    
    def weigh(self,orig,dest):
        
        ph = self.graph.getPheromEdge(orig,dest) #amount of pheromone
        des = 1/self.graph.getCostEdge(orig,dest) #desirability of the edge based on the inverse of the distance
        return ph**self.alpha + des**self.beta
    
    def make_move(self,dest):
        
        orig = self.city
        
        if dest is None:
            if self.in_nest is True:
                return None
            dest = self.start_city
        else:
            self.available_cities.remove(dest)
        
        edge = self.graph.edges[(orig,dest)]
        self.visited_cities.append(dest)        
        self.city = dest
        self.solution.append(edge)
        self.cost += self.graph.getCostEdge(orig,dest)
    
        return edge
    def __eq__(self, other):
        
        return self._cost == self._cost

    def __lt__(self, other):
      
        return self._cost < self._cost
            
        
    
    

#Find Best Path
class ACO:

    def __init__(self, graph, num_ants, alpha=1.0, beta=5.0, num_gen=10, rho=0.8):
        self.graph = graph
        self.num_ants = num_ants
        self.alpha = alpha # pheromon importance
        self.beta = beta # distance importance
        self.num_gen = num_gen # number of generations of the nest
        self.rho = rho # Evaporation Rate
        self.min_pherom = 0.01
        self.p_deposit = 1
        self.ants = [] # ants list
    
    # In this function, There will be at least one Ant starting at each city, as long as Number of Ants >= Number of cities
    def createColony(self):
        list_cities = [city for city in range(self.graph.num_vertices)]
        random.shuffle(list_cities) #randomize list of cities, so each gen have a different composition
        for k in range(self.num_ants):
            ant_city = list_cities[0]
            list_cities.remove(ant_city) # remove it from the list 
            list_cities.append(ant_city) # append it again at the end
            self.ants.append(Ant(self.graph,ant_city,self.alpha,self.beta))
    
    
    #These functions solve the Global solutions for the map. the first one return the final Global best path
    #the second one yield (return) each time it is called the best solution thus far.
    def solveWorld(self):
        self.graph.pheromReset()
        global_best = None
        for gen in range(self.num_gen):
            self.createColony()
            local_best = self.solveGen()
            if global_best is None or local_best < global_best:
                global_best = local_best
        return global_best
    
    def solveWorld_byStep(self):
        self.graph.pheromReset()
        global_best = None        
        for gen in range(num_gen):
            self.createColony()
            local_best = self.solveGen()
            if global_best is None or local_best < global_best:
                global_best = local_best
                yield global_best
    
    def solveGen(self):
        self.find_solutions()
        self.global_update()
        return sorted(self.ants)[0]
    
    def find_solutions(self):
        
        ants_nest = 0
        while ants_nest < self.num_ants:
            for ant in self.ants:
                if not ant.in_nest():
                    edge = ant.move()
                    self.local_update(edge)
                else:
                    ants_nest += 1
    
    #this function update the pheromone value on this edge, so the next ant already see the new value,
    #this way the program mimics the natural dinamic of food search by ants
    def local_update(self, edge):
        pherom = max(self.min_pherom,edge.getPherom() * self.rho)
        edge.setPherom(pherom)
        
        
    #this function updates the pheromone value on all edges after one generation
    #the pheromone drops to (1-rho)  as a time-like effect
    #after it, the edges on the best paths gets updated accordingly to the fitness of the solution
    #the edges on the best solution get more pheromone than the edges of the second best
    #that get more pheromone than the third best solution and so on
    def global_update(self):
        ants = sorted(self.ants)
        for ant in ants:
            boost = self.p_deposit / ant.cost
            for edge in ant.path:
                pherom = max(self.min_pherom, (1-self.rho) * edge.getPherom() + boost)
                edge.setPherom(pherom)
    
    
    
            
    
    
    
    

In [335]:
from lxml import etree

xml = etree.parse('burma14.xml')
vertices = xml.findall('graph/vertex')
graph = Graph(len(vertices))
for i in range(len(vertices)):
    edges = vertices[i].findall('edge')
    for j in range(len(edges)):
        cost = float(edges[j].get('cost'))
        dest = int(edges[j].text)
        graph.addEdge(i,dest,cost)

num_ants = 3* len(vertices)
colony = ACO(graph,num_ants, alpha=3.0, beta=3.0, num_gen=10, rho=0.7)
solution = colony.solveWorld()
print(solution.cost)

6153.0


In [384]:
import pants,math,random,os

from lxml import etree


def euclidean(a, b):
    return math.sqrt(pow(a[1] - b[1], 2) + pow(a[0] - b[0], 2))

def geo(a,b):
    latitude = []
    longitude = []
    PI = 3.141592
    deg = int(a[0])
    _min = a[0]- deg
    rad = PI * (deg + 5.0 * _min/ 3.0) / 180.0
    latitude.append(rad)
    
    PI = 3.141592
    deg = int(b[0])
    _min = b[0]- deg
    rad = PI * (deg + 5.0 * _min/ 3.0) / 180.0
    latitude.append(rad)
    
    PI = 3.141592
    deg = int(a[1])
    _min = a[1]- deg
    rad = PI * (deg + 5.0 * _min/ 3.0) / 180.0
    longitude.append(rad)
    
    PI = 3.141592
    deg = int(b[1])
    _min = b[1]- deg
    rad = (PI * (deg + 5.0 * _min/ 3.0) / 180.0)
    longitude.append(rad)
    
    RRR = 6378.388
    q1 = math.cos( longitude[0] - longitude[1] )
    q2 = math.cos( latitude[0] - latitude[1] )
    q3 = math.cos( latitude[0] + latitude[1] )
    dij =  int( RRR * math.acos( 0.5*((1.0+q1)*q2 - (1.0-q1)*q3) ) + 1.0)
    return dij
    

xml = etree.parse('burma14.xml')
vertices = xml.findall('graph/vertex')
burma14 = []
burma14.append((16.47,96.10))
burma14.append((16.47,94.44))
burma14.append((20.09,92.54))
burma14.append((22.39,93.37))
burma14.append((25.23,97.24))
burma14.append((22.00,96.05))
burma14.append((20.47,97.02))
burma14.append((17.20,96.29))
burma14.append((16.30,97.38))
burma14.append((14.05,98.12))
burma14.append((16.53,97.38))
burma14.append((21.52,95.59))
burma14.append((19.41,97.13))
burma14.append((20.09,94.55))

        
        

world = pants.World(burma14,geo)
#test build
ants = [10,50,100,200,500,1000,1500]
alphas = [0,0.25,0.5,0.75,1,1.5,2,2.5,3,3.5,4,4.5,5,6,7]
betas = [0,0.25,0.5,0.75,1,1.5,2,2.5,3,3.5,4,4.5,5,6,7]
rhos = [0.2,0.4,0.5,0.6,0.8]
elites = [0,0.2,0.4,0.5,0.6,0.8]
limits = [10,100,500,1000,2000,5000,10000]

med = 0
it = len(alphas)*len(betas)*len(rhos)*len(limits)*len(ants)*len(elites)
for ant in ants:
    for alpha in alphas:
        for beta in betas:
            for rho in rhos:
                for elite in elites:
                    for limit in limits:
                        for i in range(30):
                            solver = pants.Solver(ant_count=ant,alpha=alpha,beta=beta,rho=rho,elite=elite,limit=limit)
                            solution = solver.solve(world)
                            med += solution.distance 
                            #print(solution.distance)
                            fo = open('burma14.test','a')
                            string = 'ID: {i}\tant: {0}\talpha: {1}\tbeta: {2}\trho: {3}\telite: {4}\tlimit: {5}\tdistance: {6}\n'.format(ant,alpha,beta,rho,elite,limit,solution.distance,i=i)
                            fo.write(string)
                            fo.close()
                        fo = open('burma14.test','a')
                        string = 'Med \tant: {0}\talpha: {1}\tbeta: {2}\trho: {3}\telite: {4}\tlimit: {5}\tdistance: {6}\n'.format(ant,alpha,beta,rho,elite,limit,int(med/30),i=i)
                        fo.write(string)
                        fo.close()  
                        break
                    break
                break
            break
        break
    break
print('finish')
    


5354
5183
4862
5166
4235
4904
4930
5176
5055
4998
4874
4714
4780
4259
4318
4726
4658
4582
4656
5135
4786
4601
4828
5155
4702
4677
5275
5044
4803
4998
finish


In [443]:
"""
    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.
    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.
    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""
import re
import matplotlib.pyplot as plt

#  We use a regex here to clean characters and keep only numerics


#  we open the TSP file and put each line cleaned of spaces
#  and newline characters in a list 
def read_tsp_data(tsp_name):
    tsp_name = tsp_name
    with open(tsp_name) as f:
        content = f.read().splitlines()
        cleaned = [x.lstrip() for x in content if x != ""]
        f.close()
        return cleaned
"""
We return a list like 
['NAME: ulysses16.tsp',
'TYPE: TSP',
'COMMENT: Odyssey of Ulysses (Groetschel/Padberg)',
'DIMENSION: 16',
'EDGE_WEIGHT_TYPE: GEO',
'DISPLAY_DATA_TYPE: COORD_DISPLAY',
'NODE_COORD_SECTION',
'1 38.24 20.42',
'2 39.57 26.15',
'3 40.56 25.32',
................
'EOF']
"""

"""
Check for the line DIMENSION in the file and keeps the numeric value
"""
def detect_dimension(in_list):
    non_numeric = re.compile(r'[^\d]+')
    for element in in_list:
        if element.startswith("DIMENSION"):
            return non_numeric.sub("",element)

"""
Iterate through the list of line from the file
if the line starts with a numeric value within the 
range of the dimension , we keep the rest which are
the coordinates of each city
1 33.00 44.00 results to 33.00 44.00
"""
def get_cities(list,dimension,cities_set):
    dimension = int(dimension)
    for item in list:
        for num in range(1, dimension + 1):
            if item.startswith(str(num)):
                index, space, rest = item.partition(' ')
                if rest not in cities_set:
                    cities_set.append(rest)
    return cities_set


"""
Brake each coordinate 33.00 44.00 to a tuple ('33.00','44.00')
"""
def city_tup(cities_tups,list):
    for item in list:
        first_coord, second_coord = item.split(' ')
        cities_tups.append((float(first_coord), float(second_coord)))
    return cities_tups

"""
We zip for reference each city to a number
in order to work and solve the TSP we need a list 
of cities like 
[1,2,3,4,5,...........]
with the dictionary we will have a reference of the coordinates of each city 
to calculate the distance within (i + 1, i) or (2 - 1) were 2 and 1 represents each city
"""
def create_cities_dict(cities_tups):
    return zip((range(1,len(cities_tups)+1)),cities_tups)

"""
Just to plot the results
"""
def plot_cities(cities_tups):
    plt.clf()
    plt.scatter(*zip(*cities_tups))
    plt.plot(*zip(*cities_tups))
    plt.show()


"""
Putting it all together
"""
def produce_final(file):
    cities_set = []
    cities_tups = []
    cities_dict = {}
    data = read_tsp_data(file)
    dimension = detect_dimension(data)
    cities_set = get_cities(data,dimension,cities_set)
    cities_tups = city_tup(cities_tups,cities_set)
    #cities_dict = create_cities_dict(cities_tups)
    #plot_cities(cities_tups)
    #print(cities_dict)
    return cities_tups
    
burma14 = produce_final('burma14.tsp')
berlin52 = produce_final('berlin52.tsp')
rd100 = produce_final('rd100.tsp')
tsp225 = produce_final('tsp225.tsp')
pcb442 = produce_final('pcb442.tsp')
u574 = produce_final('u574.tsp')
u1060 = produce_final('u1060.tsp')
vm1084 = produce_final('vm1084.tsp')


In [ ]:

#test build
ants = [10,50,100,200,500,1000,1500]
alphas = [0,0.25,0.5,0.75,1,1.5,2,2.5,3,3.5,4,4.5,5,6,7]
betas = [0,0.25,0.5,0.75,1,1.5,2,2.5,3,3.5,4,4.5,5,6,7]
rhos = [0.2,0.4,0.5,0.6,0.8]
elites = [0,0.2,0.4,0.5,0.6,0.8]
limits = [10,100,500,1000,2000,5000,10000]

#Burma14
world = pants.World(burma14,geo)
med = 0
for ant in ants:
    for alpha in alphas:
        for beta in betas:
            for rho in rhos:
                for elite in elites:
                    for limit in limits:
                        for i in range(30):
                            solver = pants.Solver(ant_count=ant,alpha=alpha,beta=beta,rho=rho,elite=elite,limit=limit)
                            solution = solver.solve(world)
                            med += solution.distance 
                            #print(solution.distance)
                            fo = open('burma14.test','a')
                            string = 'ID: {i}\tant: {0}\talpha: {1}\tbeta: {2}\trho: {3}\telite: {4}\tlimit: {5}\tdistance: {6}\n'.format(ant,alpha,beta,rho,elite,limit,solution.distance,i=i)
                            fo.write(string)
                            fo.close()
                        fo = open('burma14.test','a')
                        string = 'Med \tant: {0}\talpha: {1}\tbeta: {2}\trho: {3}\telite: {4}\tlimit: {5}\tdistance: {6}\n'.format(ant,alpha,beta,rho,elite,limit,int(med/30),i=i)
                        fo.write(string)
                        fo.close()


#berlin52
world = pants.World(berlin52,euclidean)
med = 0
for ant in ants:
    for alpha in alphas:
        for beta in betas:
            for rho in rhos:
                for elite in elites:
                    for limit in limits:
                        for i in range(30):
                            solver = pants.Solver(ant_count=ant,alpha=alpha,beta=beta,rho=rho,elite=elite,limit=limit)
                            solution = solver.solve(world)
                            med += solution.distance 
                            #print(solution.distance)
                            fo = open('berlin52.test','a')
                            string = 'ID: {i}\tant: {0}\talpha: {1}\tbeta: {2}\trho: {3}\telite: {4}\tlimit: {5}\tdistance: {6}\n'.format(ant,alpha,beta,rho,elite,limit,solution.distance,i=i)
                            fo.write(string)
                            fo.close()
                        fo = open('berlin52.test','a')
                        string = 'Med \tant: {0}\talpha: {1}\tbeta: {2}\trho: {3}\telite: {4}\tlimit: {5}\tdistance: {6}\n'.format(ant,alpha,beta,rho,elite,limit,int(med/30),i=i)
                        fo.write(string)
                        fo.close()

#rd100
world = pants.World(rd100,euclidean)
med = 0
for ant in ants:
    for alpha in alphas:
        for beta in betas:
            for rho in rhos:
                for elite in elites:
                    for limit in limits:
                        for i in range(30):
                            solver = pants.Solver(ant_count=ant,alpha=alpha,beta=beta,rho=rho,elite=elite,limit=limit)
                            solution = solver.solve(world)
                            med += solution.distance 
                            #print(solution.distance)
                            fo = open('rd100.test','a')
                            string = 'ID: {i}\tant: {0}\talpha: {1}\tbeta: {2}\trho: {3}\telite: {4}\tlimit: {5}\tdistance: {6}\n'.format(ant,alpha,beta,rho,elite,limit,solution.distance,i=i)
                            fo.write(string)
                            fo.close()
                        fo = open('rd100.test','a')
                        string = 'Med \tant: {0}\talpha: {1}\tbeta: {2}\trho: {3}\telite: {4}\tlimit: {5}\tdistance: {6}\n'.format(ant,alpha,beta,rho,elite,limit,int(med/30),i=i)
                        fo.write(string)
                        fo.close()

#Tsp225                        
world = pants.World(tsp225,euclidean)

med = 0
for ant in ants:
    for alpha in alphas:
        for beta in betas:
            for rho in rhos:
                for elite in elites:
                    for limit in limits:
                        for i in range(30):
                            solver = pants.Solver(ant_count=ant,alpha=alpha,beta=beta,rho=rho,elite=elite,limit=limit)
                            solution = solver.solve(world)
                            med += solution.distance 
                            #print(solution.distance)
                            fo = open('tsp225.test','a')
                            string = 'ID: {i}\tant: {0}\talpha: {1}\tbeta: {2}\trho: {3}\telite: {4}\tlimit: {5}\tdistance: {6}\n'.format(ant,alpha,beta,rho,elite,limit,solution.distance,i=i)
                            fo.write(string)
                            fo.close()
                        fo = open('tsp225.test','a')
                        string = 'Med \tant: {0}\talpha: {1}\tbeta: {2}\trho: {3}\telite: {4}\tlimit: {5}\tdistance: {6}\n'.format(ant,alpha,beta,rho,elite,limit,int(med/30),i=i)
                        fo.write(string)
                        fo.close()

#pcb442                        
world = pants.World(pcb442,euclidean)
med = 0
for ant in ants:
    for alpha in alphas:
        for beta in betas:
            for rho in rhos:
                for elite in elites:
                    for limit in limits:
                        for i in range(30):
                            solver = pants.Solver(ant_count=ant,alpha=alpha,beta=beta,rho=rho,elite=elite,limit=limit)
                            solution = solver.solve(world)
                            med += solution.distance 
                            #print(solution.distance)
                            fo = open('pcb442.test','a')
                            string = 'ID: {i}\tant: {0}\talpha: {1}\tbeta: {2}\trho: {3}\telite: {4}\tlimit: {5}\tdistance: {6}\n'.format(ant,alpha,beta,rho,elite,limit,solution.distance,i=i)
                            fo.write(string)
                            fo.close()
                        fo = open('pcb442.test','a')
                        string = 'Med \tant: {0}\talpha: {1}\tbeta: {2}\trho: {3}\telite: {4}\tlimit: {5}\tdistance: {6}\n'.format(ant,alpha,beta,rho,elite,limit,int(med/30),i=i)
                        fo.write(string)
                        fo.close()

#u574                        
world = pants.World(u574,euclidean)
med = 0
for ant in ants:
    for alpha in alphas:
        for beta in betas:
            for rho in rhos:
                for elite in elites:
                    for limit in limits:
                        for i in range(30):
                            solver = pants.Solver(ant_count=ant,alpha=alpha,beta=beta,rho=rho,elite=elite,limit=limit)
                            solution = solver.solve(world)
                            med += solution.distance 
                            #print(solution.distance)
                            fo = open('u574.test','a')
                            string = 'ID: {i}\tant: {0}\talpha: {1}\tbeta: {2}\trho: {3}\telite: {4}\tlimit: {5}\tdistance: {6}\n'.format(ant,alpha,beta,rho,elite,limit,solution.distance,i=i)
                            fo.write(string)
                            fo.close()
                        fo = open('u574.test','a')
                        string = 'Med \tant: {0}\talpha: {1}\tbeta: {2}\trho: {3}\telite: {4}\tlimit: {5}\tdistance: {6}\n'.format(ant,alpha,beta,rho,elite,limit,int(med/30),i=i)
                        fo.write(string)
                        fo.close()

#u1060                        
world = pants.World(u1060,euclidean)
med = 0
for ant in ants:
    for alpha in alphas:
        for beta in betas:
            for rho in rhos:
                for elite in elites:
                    for limit in limits:
                        for i in range(30):
                            solver = pants.Solver(ant_count=ant,alpha=alpha,beta=beta,rho=rho,elite=elite,limit=limit)
                            solution = solver.solve(world)
                            med += solution.distance 
                            #print(solution.distance)
                            fo = open('u1060.test','a')
                            string = 'ID: {i}\tant: {0}\talpha: {1}\tbeta: {2}\trho: {3}\telite: {4}\tlimit: {5}\tdistance: {6}\n'.format(ant,alpha,beta,rho,elite,limit,solution.distance,i=i)
                            fo.write(string)
                            fo.close()
                        fo = open('u1060.test','a')
                        string = 'Med \tant: {0}\talpha: {1}\tbeta: {2}\trho: {3}\telite: {4}\tlimit: {5}\tdistance: {6}\n'.format(ant,alpha,beta,rho,elite,limit,int(med/30),i=i)
                        fo.write(string)
                        fo.close()

#vm1084                        
world = pants.World(vm1084,euclidean)
med = 0
for ant in ants:
    for alpha in alphas:
        for beta in betas:
            for rho in rhos:
                for elite in elites:
                    for limit in limits:
                        for i in range(30):
                            solver = pants.Solver(ant_count=ant,alpha=alpha,beta=beta,rho=rho,elite=elite,limit=limit)
                            solution = solver.solve(world)
                            med += solution.distance 
                            #print(solution.distance)
                            fo = open('vm1084.test','a')
                            string = 'ID: {i}\tant: {0}\talpha: {1}\tbeta: {2}\trho: {3}\telite: {4}\tlimit: {5}\tdistance: {6}\n'.format(ant,alpha,beta,rho,elite,limit,solution.distance,i=i)
                            fo.write(string)
                            fo.close()
                        fo = open('vm1084.test','a')
                        string = 'Med \tant: {0}\talpha: {1}\tbeta: {2}\trho: {3}\telite: {4}\tlimit: {5}\tdistance: {6}\n'.format(ant,alpha,beta,rho,elite,limit,int(med/30),i=i)
                        fo.write(string)
                        fo.close()
                        

In [15]:
world_burma14 = 14
world_berlin52 = 5
world_rd100=43
world_tsp225=24
world_pcb442=76
world_u574=42
world_u1060=4
world_vm1084 = 3
worlds = [world_burma14, world_berlin52, world_rd100, world_tsp225, world_pcb442, world_u574, world_u1060, world_vm1084]
names = ['burma14.test','berlin52.test','rd100.test','tsp225.test','pcb442.test','u574.test','u1060.test','vm1084.test']
for index,world in enumerate(worlds):
    print(names[index])

burma14.test
berlin52.test
rd100.test
tsp225.test
pcb442.test
u574.test
u1060.test
vm1084.test
